In [1]:
from ete3 import Tree, TreeStyle
import random

In [2]:
def build_tree(file_path, index):
    with open(file_path, 'r') as file:
        parent_data = file.readlines()[index].strip().split()

    node_dict = {}

    for idx, parent_id in enumerate(parent_data, start=1):
        if idx not in node_dict:
            node_dict[idx] = Tree(name=str(idx))
        if int(parent_id) not in node_dict:
            node_dict[int(parent_id)] = Tree(name=str(parent_id))

        node_dict[int(parent_id)].add_child(node_dict[idx])
    
    return node_dict

In [3]:
def get_final_mutations_L(file_path, index):
    with open(file_path, 'r') as file:
        final_distribution = file.readlines()[index].strip().split()
    
    return [str(num) for num in final_distribution]

def get_final_mutations(file_path, index):
    with open(file_path, 'r') as file:
        final_distribution = file.readlines()[index].strip().split()
    
    return set([str(num) for num in final_distribution])

def sample_final_mutations(final_distribution, k):
    return set(random.sample(final_distribution, k))

In [4]:
def find_all_nodes(tree, nodes):
    # nodes = [str(num) for num in nodes]
    necessary_nodes = set(nodes)
    for node_id in nodes:
        current_node = tree.search_nodes(name=str(node_id))[0]
        while current_node.up: 
            necessary_nodes.add(current_node.up.name)
            current_node = current_node.up
    return necessary_nodes

In [5]:
import os
def count_non_empty_lines(file_path):
    if not os.path.exists(file_path):
        return 0

    with open(file_path, 'r') as f:
        return sum(1 for line in f if line.strip())

In [6]:
def get_coalescence_time(tree, samples):
    samples = set(samples)
    if len(samples) <= 1: return 0
    ancestor = tree.get_common_ancestor(samples)
    max_distance = max(ancestor.get_distance(tree & sample) for sample in samples)
    return max_distance

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

def calculate_coalescence(output_file, checkpoint_interval=50):
    ks = [6, 10]
    mus = [0.001]
    # mus = [0.1, 0.05, 0.01, 0.005, 0.001, 0.0001, 0.00001, 0.000001]
    ss = [0.1]#, 0.01, 0, -0.01]
    repetitions = 50 # Number of trees with same parameters
    num_samples = 100  # Number of samples per file
    iterations = 200 # Number of coalescence calculations per sample

    # Load checkpoint if it exists
    temp_file = output_file + ".tmp"
    if os.path.exists(temp_file):
        checkpoint_df = pd.read_pickle(temp_file)
        data = checkpoint_df.to_dict(orient='list')
        completed_combinations = set(zip(data['k'], data['mu'], data['s'], data['repetition']))
        print("Resuming from checkpoint.")
    else:
        data = {'k': [], 'mu': [], 's': [], 'repetition': [],
                'mean_coalescence_time_20': [], 'variance_coalescence_time_20': [],
                'mean_coalescence_time_40': [], 'variance_coalescence_time_40': [],
                'mean_coalescence_time_60': [], 'variance_coalescence_time_60': [],
                'mean_coalescence_time_80': [], 'variance_coalescence_time_80': [],
                'mean_coalescence_time_100': [], 'variance_coalescence_time_100': [],}
        completed_combinations = set()

    total_combinations = len(ks) * len(mus) * len(ss) * repetitions
    remaining_combinations = total_combinations - len(completed_combinations)

    with tqdm(total=remaining_combinations, desc='Calculating Coalescence Times') as pbar:
        count = 0
        for k in ks:
            for mu in mus:
                for s in ss:
                    for i in range(repetitions):

                        if (k, mu, s, i) in completed_combinations:
                            continue

                        tree_path = f"result_12.11/{k}_regular_{i}_mu{mu}_s{s}_tree.txt"
                        final_path = f"result_12.11/{k}_regular_{i}_mu{mu}_s{s}_list.txt"

                        if not os.path.exists(tree_path) or not os.path.exists(final_path):
                            print(f"Skipping missing paths: {tree_path}, {final_path}")
                            pbar.update(1)
                            continue

                        # Load the tree files
                        tree_samples = count_non_empty_lines(tree_path)
                        final_samples = count_non_empty_lines(final_path)
                        num_samples = min(tree_samples, final_samples, num_samples)

                        if num_samples == 0:
                            print(f"Skipping empty files: {tree_path}, {final_path}")
                            pbar.update(1)
                            continue

                        # Calculate the coalescence time
                        sample_coalescence_times = []
                        for sample_idx in range(num_samples):
                            tree_dict = build_tree(tree_path, sample_idx)
                            tree = tree_dict[0]
                            distribution = get_final_mutations_L(final_path, sample_idx)
                            path = find_all_nodes(tree, get_final_mutations(final_path, sample_idx))
                            tree.prune(path)
                            coalescence_times = []
                            for _ in range(iterations):
                                try:
                                    coalescence_times.append(get_coalescence_time(tree, random.sample(distribution, 2)))
                                except:
                                    pass
                            sample_avg_time = np.mean(coalescence_times)
                            sample_coalescence_times.append(sample_avg_time)
                            # print(sample_idx)

                        mean_coalescence_time = np.mean(sample_coalescence_times)
                        variance_coalescence_time = np.var(sample_coalescence_times)

                        data['k'].append(k)
                        data['mu'].append(mu)
                        data['s'].append(s)
                        data['repetition'].append(i)
                        data['mean_coalescence_time_20'].append(np.mean(sample_coalescence_times[:min(20, num_samples)]))
                        data['variance_coalescence_time_20'].append(np.var(sample_coalescence_times[:min(20, num_samples)]))

                        data['mean_coalescence_time_40'].append(np.mean(sample_coalescence_times[:min(40, num_samples)]))
                        data['variance_coalescence_time_40'].append(np.var(sample_coalescence_times[:min(40, num_samples)]))

                        data['mean_coalescence_time_60'].append(np.mean(sample_coalescence_times[:min(60, num_samples)]))
                        data['variance_coalescence_time_60'].append(np.var(sample_coalescence_times[:min(60, num_samples)]))

                        data['mean_coalescence_time_80'].append(np.mean(sample_coalescence_times[:min(80, num_samples)]))
                        data['variance_coalescence_time_80'].append(np.var(sample_coalescence_times[:min(80, num_samples)]))

                        data['mean_coalescence_time_100'].append(np.mean(sample_coalescence_times[:min(100, num_samples)]))
                        data['variance_coalescence_time_100'].append(np.var(sample_coalescence_times[:min(100, num_samples)]))

                        pbar.update(1)
                        count += 1

                        if count % checkpoint_interval == 0:
                            checkpoint_df = pd.DataFrame(data)
                            checkpoint_df.to_pickle(output_file + ".tmp")
                            print(f"Checkpoint saved at iteration {count}")

    df = pd.DataFrame(data)
    df.to_pickle(output_file)
    print("Final results saved.")

calculate_coalescence("coalescence_regular_50_2.pkl", checkpoint_interval=50)


In [ ]:
def get_sackin_index(tree):
    leaves = tree.get_leaves()  
    total_depth = sum(leaf.get_distance(tree) for leaf in leaves)  
    num_leaves = len(leaves)  
    if num_leaves > 1:
        normalized_index = total_depth / (0.5 * num_leaves * (num_leaves + 1) - 1)
    else:
        normalized_index = 0  
    
    # # tree depth
    # tree_depth = max(leaf.get_distance(tree) for leaf in leaves) if leaves else 0
    
    # # tree width
    # levels = {}
    # for leaf in leaves:
    #     level = leaf.get_distance(tree, topology_only=True)
    #     if level not in levels:
    #         levels[level] = 0
    #     levels[level] += 1
    # tree_width = max(levels.values()) if levels else 0

    return normalized_index

In [ ]:
def calculate_sackin(output_file, checkpoint_interval=50):
    ks = [1, 6, 11, 15, 18]
    mus = [1, 0.1, 0.01, 0.001]
    # mus = [0.1, 0.05, 0.01, 0.005, 0.001, 0.0001, 0.00001, 0.000001]
    ss = [0.1]#, 0.01, 0, -0.01]
    repetitions = 1
    num_samples = 100  # Number of samples per file

    temp_file = output_file + ".tmp"
    if os.path.exists(temp_file):
        checkpoint_df = pd.read_pickle(temp_file)
        data = checkpoint_df.to_dict(orient='list')
        completed_combinations = set(zip(data['k'], data['mu'], data['s'], data['repetition']))
        print("Resuming from checkpoint.")
    else:
        data = {'k': [], 'mu': [], 's': [], 'repetition': [],
                'sackin': [], 'sackin_variance': []}
        completed_combinations = set()

    total_combinations = len(ks) * len(mus) * len(ss) * repetitions
    remaining_combinations = total_combinations - len(completed_combinations)

    with tqdm(total=remaining_combinations, desc='Calculating Coalescence Times') as pbar:
        count = 0
        for k in ks:
            for mu in mus:
                for s in ss:
                    for i in range(repetitions):

                        if (k, mu, s, i) in completed_combinations:
                            continue
                        
                        tree_path = f"result_12.4/test_{k}_mu{mu}_s{s}_tree.txt"
                        final_path = f"result_12.4/test_{k}_mu{mu}_s{s}_list.txt"

                        if not os.path.exists(tree_path) or not os.path.exists(final_path):
                            print(f"Skipping missing paths: {tree_path}, {final_path}")
                            pbar.update(1)
                            continue

                        # Load the tree file
                        tree_samples = count_non_empty_lines(tree_path)
                        final_samples = count_non_empty_lines(final_path)
                        num_samples = min(tree_samples, final_samples, num_samples)

                        if num_samples == 0:
                            print(f"Skipping empty files: {tree_path}, {final_path}")
                            pbar.update(1)
                            continue

                        # Calculate Sackin index
                        sackinL = []
                        for sample_idx in range(num_samples):
                            tree_dict = build_tree(tree_path, sample_idx)
                            tree = tree_dict[0]
                            distribution = get_final_mutations_L(final_path, sample_idx)
                            path = find_all_nodes(tree, get_final_mutations(final_path, sample_idx))
                            tree.prune(path)
                            sackin_index = get_sackin_index(tree)
                            sackinL.append(sackin_index)
                            # print(sample_idx)

                        data['k'].append(k)
                        data['mu'].append(mu)
                        data['s'].append(s)
                        data['repetition'].append(i)
                        data['sackin'].append(np.mean(sackinL))
                        data['sackin_variance'].append(np.var(sackinL))

                        pbar.update(1)
                        count += 1

                        if count % checkpoint_interval == 0:
                            checkpoint_df = pd.DataFrame(data)
                            checkpoint_df.to_pickle(output_file + ".tmp")
                            print(f"Checkpoint saved at iteration {count}")

    df = pd.DataFrame(data)
    df.to_pickle(output_file)
    print("Final results saved.")

calculate_sackin("sackin_geoN.pkl", checkpoint_interval=50)


In [9]:
from ete3 import Tree
import numpy as np

def compute_subtree_sizes(node):
    """
    Compute S_i (total subtree size including root) and S*_i (excluding root).
    Assigns these values to the node attributes.
    """
    if node.is_leaf():
        node.S_i = 1  # A leaf has size 1 (itself)
        node.S_star_i = 0  # No subtree without root
        return 1, 0  

    subtree_size = 1  # Count itself
    child_sizes = []

    for child in node.children:
        child_s, _ = compute_subtree_sizes(child)
        child_sizes.append(child_s)
        subtree_size += child_s  # Accumulate child subtree sizes
    
    S_star_i = subtree_size - 1  # Exclude root itself
    
    # Store values as attributes inside the node
    node.S_i = subtree_size
    node.S_star_i = S_star_i
    node.child_sizes = child_sizes  # Store sizes of children

    return subtree_size, S_star_i


def compute_balance_scores(node):
    if node.is_leaf() or len(node.children) < 2:
        node.W_i = 0
        return 0
    
    p_ij = np.array([child.S_i / node.S_star_i for child in node.children])
    
    W_i_1 = -np.sum(p_ij * np.log(p_ij) / np.log(len(node.children)))
    
    node.W_i = W_i_1
    return W_i_1

def compute_normalized_balance_index(root):
    """
    Compute the normalized tree balance index J^1.
    """
    internal_nodes = [n for n in root.traverse() if not n.is_leaf()]
    
    S_star_sum = sum(n.S_star_i for n in internal_nodes)
    weighted_sum = sum(n.S_star_i / n.S_i * n.W_i for n in internal_nodes)
    
    return weighted_sum / S_star_sum if S_star_sum > 0 else 0

k = 3
i = 0
mu = 0.001
s = 0
tree_path = f"graphs/results1/{k}_regular_graph/{k}_regular_graph_{i}_mu{mu}_s{s}_tree.txt"
final_path = f"graphs/results1/{k}_regular_graph/{k}_regular_graph_{i}_mu{mu}_s{s}_list.txt"
tree_dict = build_tree(tree_path, 0)


# Assuming root node is labeled as '1'
root = tree_dict[0]

# Step 1: Compute sizes
compute_subtree_sizes(root)

# Step 2: Compute balance scores
for node in root.traverse():
    compute_balance_scores(node)

# Step 3: Compute the normalized balance index
J1 = compute_normalized_balance_index(root)

print("Normalized Tree Balance Index (J1):", J1)


Normalized Tree Balance Index (J1): 0.004066544747433984


In [12]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

def calculate_coalescence_df_with_progress_and_saves(output_file, checkpoint_interval=50):
    """Calculate mean and variance of coalescence times with progress bar and periodic saves.

    Args:
        output_file (str): Path to the final output file (Pandas DataFrame saved as a pickle file).
        checkpoint_interval (int): Number of iterations after which to save a temporary file.
    """
    ks = [3, 4, 6, 10]
    mus = [0.001]
    # mus = [0.1, 0.05, 0.01, 0.005, 0.001, 0.0001, 0.00001, 0.000001]
    ss = [0.1]#, 0.01, 0, -0.01]
    repetitions = 50
    num_samples = 100  # Number of samples per file
    iterations = 200 # Number of coalescence calculations per sample

    # Load checkpoint if it exists
    temp_file = output_file + ".tmp"
    if os.path.exists(temp_file):
        checkpoint_df = pd.read_pickle(temp_file)
        data = checkpoint_df.to_dict(orient='list')
        completed_combinations = set(zip(data['k'], data['mu'], data['s'], data['repetition']))
        print("Resuming from checkpoint.")
    else:
        data = {'k': [], 'mu': [], 's': [], 'repetition': [],
                'Js': [], 'variance_Js': [],}
        completed_combinations = set()

    # Calculate the total number of combinations for the progress bar
    total_combinations = len(ks) * len(mus) * len(ss) * repetitions
    remaining_combinations = total_combinations - len(completed_combinations)

    # Initialize progress bar
    with tqdm(total=remaining_combinations, desc='Calculating Coalescence Times') as pbar:
        count = 0
        for k in ks:
            for mu in mus:
                for s in ss:
                    for i in range(repetitions):
                        # Skip if this combination has already been completed
                        if (k, mu, s, i) in completed_combinations:
                            continue

                        tree_path = f"graphs/result_11.13/{k}_regular_{i}_mu{mu}_s{s}_tree.txt"
                        final_path = f"graphs/result_11.13/{k}_regular_{i}_mu{mu}_s{s}_list.txt"

                        # Check if paths exist; skip if they don't
                        if not os.path.exists(tree_path) or not os.path.exists(final_path):
                            print(f"Skipping missing paths: {tree_path}, {final_path}")
                            pbar.update(1)
                            continue

                        # Load the tree file (assuming the sample index isn't needed)
                        tree_samples = count_non_empty_lines(tree_path)
                        final_samples = count_non_empty_lines(final_path)
                        num_samples = min(tree_samples, final_samples, num_samples)

                        if num_samples == 0:
                            print(f"Skipping empty files: {tree_path}, {final_path}")
                            pbar.update(1)
                            continue

                        # Calculate the coalescence time for each sample in the file
                        Js = []
                        for sample_idx in range(num_samples):
                            tree_dict = build_tree(tree_path, sample_idx)
                            tree = tree_dict[0]

                            compute_subtree_sizes(tree)

                            for node in tree.traverse():
                                compute_balance_scores(node)

                            J1 = compute_normalized_balance_index(tree)

                            Js.append(J1)
                            # print(sample_idx)

                        # Calculate mean and variance across all samples
                        mean_Js = np.mean(Js)
                        variance_Js = np.var(Js)

                        # Append results to the data dictionary
                        data['k'].append(k)
                        data['mu'].append(mu)
                        data['s'].append(s)
                        data['repetition'].append(i)
                        data['Js'].append(np.mean(mean_Js))
                        data['variance_Js'].append(np.var(mean_Js))


                        # Update progress bar after each combination
                        pbar.update(1)
                        count += 1

                        # Save a checkpoint after every `checkpoint_interval` iterations
                        if count % checkpoint_interval == 0:
                            checkpoint_df = pd.DataFrame(data)
                            checkpoint_df.to_pickle(output_file + ".tmp")
                            print(f"Checkpoint saved at iteration {count}")

    # Final save: Create a DataFrame and save as the final file
    df = pd.DataFrame(data)
    df.to_pickle(output_file)
    print("Final results saved.")

# Call the function to write the stats to a pandas DataFrame file with checkpointing
calculate_coalescence_df_with_progress_and_saves("Js_regular_50.pkl", checkpoint_interval=50)


Calculating Coalescence Times:  25%|██▌       | 50/200 [06:46<22:34,  9.03s/it]

Checkpoint saved at iteration 50


Calculating Coalescence Times:  50%|█████     | 100/200 [13:36<13:04,  7.84s/it]

Checkpoint saved at iteration 100


Calculating Coalescence Times:  75%|███████▌  | 150/200 [20:17<06:41,  8.04s/it]

Checkpoint saved at iteration 150


Calculating Coalescence Times: 100%|██████████| 200/200 [26:55<00:00,  8.08s/it]

Checkpoint saved at iteration 200
Final results saved.


In [15]:
import pandas as pd
df31 = pd.read_pickle("Js_regular_50.pkl")
df31

,k,mu,s,repetition,Js,variance_Js
0,3,0.001,0.1,0,0.000323,0.0
1,3,0.001,0.1,1,0.000327,0.0
2,3,0.001,0.1,2,0.000328,0.0
3,3,0.001,0.1,3,0.000331,0.0
4,3,0.001,0.1,4,0.000335,0.0
...,...,...,...,...,...,...
195,10,0.001,0.1,45,0.000242,0.0
196,10,0.001,0.1,46,0.000242,0.0
197,10,0.001,0.1,47,0.000243,0.0
198,10,0.001,0.1,48,0.000250,0.0
